### Prompting Chaining with News 

In [13]:

#=======================#
# ---- libraries ----- #
#=======================#

from pydantic import BaseModel, Field, validator
from typing import List, Optional, Dict
from datetime import datetime, date
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import time
import re
from enum import Enum
from openai import OpenAI  

In [5]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [6]:
#========================#
# ---- model design ---- #
#========================#


class OpenAIModels(str, Enum):
    GPT_4O_MINI = "gpt-4o-mini"
    GPT_41_MINI = "gpt-4.1-mini"
    GPT_41_NANO = "gpt-4.1-nano"


MODEL = OpenAIModels.GPT_41_NANO


In [7]:
#==========================#
# ----- Agent System ----- #
#==========================#

def get_completion(messages=None, 
                   system_prompt=None, 
                   user_prompt=None, 
                   model=MODEL):
    """
    Function to get a completion from the OpenAI API.
    Args:
        system_prompt: The system prompt
        user_prompt: The user prompt
        model: The model to use (default is gpt-4.1-mini)
    Returns:
        The completion text
    """

    messages = list(messages)
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})
    if user_prompt:
        messages.append({"role": "user", "content": user_prompt})
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.000001,
    )
    return response.choices[0].message.content

In [11]:
#================================#
# ---- Pydantic Models ---- #
#================================#

class ArticleSummary(BaseModel):
    """Resultado del Paso 1: Resumen"""
    article_id: str
    summary: str
    key_points: List[str]
    
    @validator('summary')
    def validate_summary(cls, v):
        if len(v) < 50:
            raise ValueError('Resumen muy corto (mínimo 50 caracteres)')
        return v
    
    @validator('key_points')
    def validate_points(cls, v):
        if len(v) < 1:
            raise ValueError('Debe haber al menos 1 punto clave')
        return v


class ArticleTopic(BaseModel):
    """Resultado del Paso 2: Identificación de topic"""
    article_id: str
    primary_topic: str
    categories: List[str]
    
    @validator('primary_topic')
    def validate_topic(cls, v):
        if not v or len(v) < 3:
            raise ValueError('Topic principal no identificado correctamente')
        return v


class ArticleRelevance(BaseModel):
    """Resultado del Paso 3: Calificación de relevancia"""
    article_id: str
    relevance_score: float
    impact_level: str
    reasoning: str
    urgency: str
    
    @validator('relevance_score')
    def validate_score(cls, v):
        if not (0 <= v <= 10):
            raise ValueError('Score debe estar entre 0 y 10')
        return v


class FinalRanking(BaseModel):
    """Resultado del Paso 4: Ranking final"""
    most_important_id: str
    ranking: List[Dict]
    executive_summary: str
    
    @validator('most_important_id')
    def validate_important(cls, v):
        if not v:
            raise ValueError('No se identificó la noticia más importante')
        return v

/var/folders/11/06r7d20d0z1df3fwdm3jl16c0000gn/T/ipykernel_25049/263731771.py:11: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('summary')
/var/folders/11/06r7d20d0z1df3fwdm3jl16c0000gn/T/ipykernel_25049/263731771.py:17: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('key_points')
/var/folders/11/06r7d20d0z1df3fwdm3jl16c0000gn/T/ipykernel_25049/263731771.py:30: PydanticDeprecatedSince20: Pydantic V1 style `@

In [ ]:
#================================#
# ---- Helper Functions ---- #
#================================#

def extract_date_from_url(url: str) -> Optional[date]:
    """
    Extrae fecha de URL formato: /YYYY/MM/DD/
    
    Returns:
        date object o None si no se encuentra
    """
    pattern = r'/(\d{4})/(\d{2})/(\d{2})/'
    match = re.search(pattern, url)
    
    if match:
        year, month, day = map(int, match.groups())
        try:
            return date(year, month, day)
        except ValueError:
            return None
    return None


def is_today(article_date: Optional[date]) -> bool:
    """
    Compara fecha del artículo con HOY (no hardcoded)
    
    Returns:
        True si es de hoy, False en caso contrario
    """
    if not article_date:
        return False
    
    today = date.today()  
    return article_date == today


#================================#
# ---- Gate Check Functions ---- #
#================================#

def gate_check_date(article_dict: dict) -> tuple[bool, str]:
    """
    GATE CHECK 1: Verificar que sea noticia de HOY
    
    Returns:
        (es_valido, mensaje_error)
    """
    url = article_dict.get('url', '')
    article_date = extract_date_from_url(url)
    
    if not article_date:
        return False, "No se pudo extraer fecha de la URL"
    
    if not is_today(article_date):
        today_str = date.today().strftime('%d/%m/%Y')
        article_str = article_date.strftime('%d/%m/%Y')
        return False, f"Noticia no es de hoy. Fecha: {article_str}, Hoy: {today_str}"
    
    return True, "✓ Fecha válida"


def gate_check_content_quality(article_dict: dict) -> tuple[bool, List[str]]:
    """
    GATE CHECK 2: Verificar calidad del contenido
    
    Validaciones:
    - Título >= 10 caracteres
    - Contenido >= 200 caracteres
    - Palabras >= 50
    - Sin HTML residual
    - Estructura de párrafos (puntos)
    
    Returns:
        (es_valido, lista_de_errores)
    """
    errors = []
    
    # Check: Título
    title = article_dict.get('title', '')
    if len(title) < 5:
        errors.append(f"Título muy corto: {len(title)} caracteres")
    
    # Check: Contenido
    content = article_dict.get('content', '')
    if len(content) < 200:
        errors.append(f"Contenido muy corto: {len(content)} caracteres (mínimo 200)")
    
    # Check: Palabras
    word_count = len(content.split())
    if word_count < 50:
        errors.append(f"Pocas palabras: {word_count} (mínimo 50)")
    
    # Check: HTML residual
    if content.count('<') > 3 or content.count('>') > 3:
        errors.append("Contenido tiene HTML sin procesar")
    
    # Check: Estructura de párrafos
    if content.count('.') < 2:
        errors.append("Contenido no tiene estructura de párrafos")
    
    is_valid = len(errors) == 0
    return is_valid, errors



In [15]:
#================================#
# ---- Gate Check Functions ---- #
#================================#

def gate_check_date(article_dict: dict) -> tuple[bool, str]:
    """
    GATE CHECK 1: Verificar que sea noticia de HOY
    
    Returns:
        (es_valido, mensaje_error)
    """
    url = article_dict.get('url', '')
    article_date = extract_date_from_url(url)
    
    if not article_date:
        return False, "No se pudo extraer fecha de la URL"
    
    if not is_today(article_date):
        today_str = date.today().strftime('%d/%m/%Y')
        article_str = article_date.strftime('%d/%m/%Y')
        return False, f"Noticia no es de hoy. Fecha: {article_str}, Hoy: {today_str}"
    
    return True, "✓ Fecha válida"


def gate_check_content_quality(article_dict: dict) -> tuple[bool, List[str]]:
    """
    GATE CHECK 2: Verificar calidad del contenido
    
    Validaciones:
    - Título >= 10 caracteres
    - Contenido >= 200 caracteres
    - Palabras >= 50
    - Sin HTML residual
    - Estructura de párrafos (puntos)
    
    Returns:
        (es_valido, lista_de_errores)
    """
    errors = []
    
    # Check: Título
    title = article_dict.get('title', '')
    if len(title) < 10:
        errors.append(f"Título muy corto: {len(title)} caracteres")
    
    # Check: Contenido
    content = article_dict.get('content', '')
    if len(content) < 200:
        errors.append(f"Contenido muy corto: {len(content)} caracteres (mínimo 200)")
    
    # Check: Palabras
    word_count = len(content.split())
    if word_count < 50:
        errors.append(f"Pocas palabras: {word_count} (mínimo 50)")
    
    # Check: HTML residual
    if content.count('<') > 3 or content.count('>') > 3:
        errors.append("Contenido tiene HTML sin procesar")
    
    # Check: Estructura de párrafos
    if content.count('.') < 2:
        errors.append("Contenido no tiene estructura de párrafos")
    
    is_valid = len(errors) == 0
    return is_valid, errors

In [16]:
#========================================#
# ---- Web Scraper with Gate Checks ---- #
#========================================#

def scrape_cnn_colombia(max_articles: int = 10) -> tuple[List[dict], List[dict]]:
    """
    Scraper de CNN Español Colombia con gate checks integrados
    
    Returns:
        (articulos_validos, articulos_rechazados)
    """
    base_url = 'https://cnnespanol.cnn.com/colombia'
    
    print("="*80)
    print("🗞️  SCRAPING CNN ESPAÑOL - COLOMBIA")
    print("="*80)
    print(f"📅 Buscando noticias de HOY: {date.today().strftime('%A, %d de %B de %Y')}")
    print(f"🎯 Máximo artículos: {max_articles}")
    print("="*80 + "\n")
    
    valid_articles = []  # ⭐ Lista para artículos válidos
    rejected_articles = []  # ⭐ Lista para artículos rechazados
    
    try:
        # Descargar página principal
        response = requests.get(base_url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontrar todos los enlaces de artículos
        article_links = []
        seen_urls = set()
        
        for link in soup.find_all('a', class_='container__link', href=True):
            href = link.get('href', '')
            
            # Construir URL completa
            if not href.startswith('http'):
                href = f"https://cnnespanol.cnn.com{href}"
            
            # Extraer título
            title_span = link.find('span', class_='container__headline-text')
            
            if title_span and href not in seen_urls:
                title = title_span.get_text(strip=True)
                article_links.append({'title': title, 'url': href})
                seen_urls.add(href)
                
                if len(article_links) >= max_articles:
                    break
        
        print(f"✅ Encontrados {len(article_links)} enlaces de artículos\n")
        print("📥 Descargando contenido y aplicando gate checks...\n")
        
        # Procesar cada artículo
        for i, article_data in enumerate(article_links, 1):
            print(f"[{i}/{len(article_links)}] {article_data['title'][:60]}...")
            
            # GATE CHECK 1: Verificar fecha en URL
            is_valid_date, date_msg = gate_check_date(article_data)
            
            if not is_valid_date:
                print(f"   ⏰ RECHAZADO - {date_msg}")
                rejected_articles.append({
                    **article_data,
                    'rejection_reason': 'fecha_invalida',
                    'rejection_detail': date_msg
                })
                time.sleep(0.3)
                continue
            
            # Descargar contenido del artículo
            try:
                article_response = requests.get(article_data['url'], timeout=10)
                article_response.raise_for_status()
                article_soup = BeautifulSoup(article_response.content, 'html.parser')
                
                # Extraer párrafos
                paragraphs = article_soup.find_all('p', class_='paragraph')
                content = "\n\n".join([p.get_text(strip=True) for p in paragraphs])
                
                if not content:
                    print(f"   ❌ RECHAZADO - Sin contenido")
                    rejected_articles.append({
                        **article_data,
                        'rejection_reason': 'sin_contenido',
                        'rejection_detail': 'No se encontraron párrafos'
                    })
                    time.sleep(0.3)
                    continue
                
                # Agregar contenido al dict
                article_data['content'] = content
                article_data['scraped_at'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                article_data['word_count'] = len(content.split())
                
                # GATE CHECK 2: Calidad de contenido
                is_valid_quality, quality_errors = gate_check_content_quality(article_data)
                
                if not is_valid_quality:
                    print(f"   ⚠️  RECHAZADO - {'; '.join(quality_errors)}")
                    rejected_articles.append({
                        **article_data,
                        'rejection_reason': 'calidad_insuficiente',
                        'rejection_detail': quality_errors
                    })
                    time.sleep(0.3)
                    continue
                
                # Artículo VÁLIDO
                valid_articles.append(article_data)
                print(f"   ✅ VÁLIDO - {article_data['word_count']} palabras - {date_msg}")
                
            except Exception as e:
                print(f"   ❌ ERROR - {str(e)[:50]}")
                rejected_articles.append({
                    **article_data,
                    'rejection_reason': 'error_descarga',
                    'rejection_detail': str(e)
                })
            
            time.sleep(0.5)  # Rate limiting
        
        # Resumen final
        print("\n" + "="*80)
        print("📊 RESUMEN DEL SCRAPING")
        print("="*80)
        print(f"Total procesados: {len(article_links)}")
        print(f"✅ Artículos válidos: {len(valid_articles)}")
        print(f"❌ Artículos rechazados: {len(rejected_articles)}")
        print(f"📈 Tasa de éxito: {len(valid_articles)/max(len(article_links), 1)*100:.1f}%")
        
        return valid_articles, rejected_articles
    
    except Exception as e:
        print(f"\n❌ ERROR FATAL en scraping: {e}")
        return [], []

In [17]:
scrape_cnn_colombia()

🗞️  SCRAPING CNN ESPAÑOL - COLOMBIA
📅 Buscando noticias de HOY: Monday, 05 de January de 2026
🎯 Máximo artículos: 10

✅ Encontrados 10 enlaces de artículos

📥 Descargando contenido y aplicando gate checks...

[1/10] "Es como un respiro, como si algo te soltara": el testimonio...
   ❌ RECHAZADO - Sin contenido
[2/10] Estos son los países a los que Trump lanzó advertencias tras...
   ✅ VÁLIDO - 1158 palabras - ✓ Fecha válida
[3/10] Petro habla de "tomar las armas" para defender la soberanía ...
   ✅ VÁLIDO - 269 palabras - ✓ Fecha válida
[4/10] Tras los ataques de EE.UU. en Venezuela, ¿qué pueden esperar...
   ✅ VÁLIDO - 1366 palabras - ✓ Fecha válida
[5/10] “Su castigo es tratarme falsamente de narcotraficante”: Petr...
   ⏰ RECHAZADO - Noticia no es de hoy. Fecha: 04/01/2026, Hoy: 05/01/2026
[6/10] Tras la captura de Maduro, Tump advierte a Petro, presidente...
   ⏰ RECHAZADO - Noticia no es de hoy. Fecha: 03/01/2026, Hoy: 05/01/2026
[7/10] Al menos 3 muertos y 4 detenidos deja operaci

([{'title': 'Estos son los países a los que Trump lanzó advertencias tras el ataque en Venezuela',
   'url': 'https://cnnespanol.cnn.com/2026/01/05/eeuu/paises-trump-advertencias-ataque-venezuela-trax',
   'content': 'Desde que las fuerzas de Estados Unidos capturaron al presidente de Venezuela, Nicolás Maduro, durante el fin de semana, el presidente Donald Trump y miembros de su Gobierno han emitido advertencias a varios países y territorios, entre ellos Colombia, Cuba, México, Irán y Groenlandia, un territorio autónomo de Dinamarca.\n\nTrump dijo el domingo: “Nuestro objetivo es tener países a nuestro alrededor que sean viables y exitosos y donde se permita que el petróleo salga libremente”.\n\n“El dominio estadounidense en el hemisferio occidental no volverá a ser cuestionado”, afirmó Trump.\n\nEsto es lo que hay que saber sobre lo que Trump ha dicho en los últimos dos días y cómo han respondido algunos de esos Gobiernos.\n\nTrump reiteró el domingo que Estados Unidos necesita la en